In [1]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
from ipywidgets import Video, Image
from glob import glob
import os
import time
import cv2
import time
import matplotlib.pyplot as plt
import json
import subprocess, shlex
import PIL as pil
import random
import functools
import pandas as pd

In [2]:
def find_frame_at_given_time(frame_num, video_path):
    
    if 'GRAHAN' in video_path:
        video_path    = '/'.join(video_path.split('/')[0:-2])+'/GRAHAN/'+'_'.join(video_path.split('/')[-1].split('_')[0:2])+'.mp4'
        
    print(video_path)
    vid_cap           = cv2.VideoCapture(video_path)
    fps               = vid_cap.get(cv2.CAP_PROP_FPS)
    total_frame       = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_num         = max(0, frame_num-1)
#     print(frame_num, total_frame)
    
    vid_cap.set(1, frame_num)
    _, frame = vid_cap.read()
    
#     print(frame.shape)
    
    return frame


def read_json(json_path):
    with open(json_path, 'rb') as f:
        return json.load(f)
    
def add_ratio_button(display_color, display_pattern):
    gender = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'}, 
        description='Gender:',
        disabled=False
    )

        
    category = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None, 
        layout={'width': 'max-content'},
        description='Category:',
        disabled=False
    )

    if display_color:
        color = widgets.RadioButtons(
            options=['good', 'bad'],
            value=None, 
            layout={'width': 'max-content'}, 
            description='Color :',
            disabled=False
        )
    else:
        color = widgets.RadioButtons(
            options=['good', 'bad', 'NO'],
            value='NO', 
            layout={'width': 'max-content'}, 
            description='Color :',
            disabled=False
        )
        

    if display_pattern:
        pattern = widgets.RadioButtons(
            options=['good', 'bad'],
            value=None,
            layout={'width': 'max-content'},
            description='Pattern:',
            disabled=False
        )   
        
    else:
        pattern = widgets.RadioButtons(
            options=['good', 'bad', 'NO'],
            value='NO',
            layout={'width': 'max-content'},
            description='Pattern:',
            disabled=False
        )   

    frame = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'},
        description='Frame:',
        disabled=False
    ) 
        
    return frame, gender, category, color, pattern

def analysis(json_path, result_path, qa_status_path, username): #image array    
    hold = False
    
    def get_qa_status(qa_status_path): #get paths from qa_status.txt
        with open(qa_status_path, 'r') as f:
            size = os.path.getsize(qa_status_path)
            if size != 0:
                return  [i.replace('\n', '') for i in f.readlines()]

            else:
                return []

    def next_item(): 
        clear_output()
        analysis(new_json, result_path, qa_status_path, username)
        
        
    def click_submit(arg, argument):

        frame, gender, category, color, pattern, object_name = arg
        
        if gender.value and color.value and category.value and pattern.value and frame.value is not None:
            with open(result_path, 'a') as f:
                f.write(f"{object_name},{frame.value},{gender.value},{category.value},{color.value},{pattern.value},{username}\n")
            print("recorded")
            argument.disabled = True
            
        else:
            print('please fill up all section')  
            
            
    def Done(passed_arg, arg):
        with open(qa_status_path, 'a') as f:
            f.write(passed_arg+'\n')
        next_item()
        
            
    def display_adds(item, object_name, series_name, video_path, display_color, display_pattern):
        global new_json, result_path
        #radio buttons
        frame, gender, category, color, pattern = add_ratio_button(display_color, display_pattern)

       

        frame_num  = int(item['features']['maxConfFrameNumber'])
        
        if display_color and display_pattern:
            img_path   = f'{images_root_path}/{series_name}/multi_color_image_matching/{object_name}/*'       
            img_path   = glob(img_path)[0]
            
        
        elif display_color and not(display_pattern):
            img_path   = f'{images_root_path}/{series_name}/multi_color/{object_name}/*'
            img_path   = random.shuffle(glob(img_path))[0]
            
        elif not(display_color):
            img_path   = f'{detected_path}/{object_name}'
                        
        final_image  = cv2.resize(cv2.imread(img_path), (300, 300))
    
        frame_       = cv2.resize(find_frame_at_given_time(frame_num, video_path), (300, 300))
    
        
        concated     = cv2.hconcat([frame_, final_image])
        cv2.imwrite('temp_file.png', concated)
        video       = Image.from_file('temp_file.png',  width = 600, height=600)
        
        #submit button
        arg    = frame, gender, category, color, pattern, object_name
        submit = widgets.Button(description = 'Submit', disabled=False)
        submit.on_click(functools.partial(click_submit, arg))
        
        #display 
        print("--> "+ img_path.split('/')[-1])
        print("--> "+ img_path.split('/')[-2])
        content = widgets.HBox([video])
        if display_color and display_pattern:
            radio = widgets.HBox([frame, gender, category, color, pattern])
            
        elif display_color and not(display_pattern):
            radio = widgets.HBox([frame, gender, category, color])
            
        elif not(display_color):
            radio = widgets.HBox([frame, gender, category])
            
    
        display(widgets.VBox([content, radio, submit, widgets.Output()]))
        
        
        

    start          = time.time()
    qa_completed   = get_qa_status(qa_status_path)  
    new_json       = list(set(json_path) - set(qa_completed))
    
    print(len(new_json))
    #If there is no json_left to evaluate task is finished
    if len(new_json) == 0:
        print("CONGRATS...TASK FINITO !!")
        return 0
    
    
    rand_seed  = np.random.choice(range(0, 10))
    np.random.seed(rand_seed)
    rand_index = np.random.choice(range(0, len(new_json)))
    
    selected_json_path  = new_json[rand_index]
    selected_json       = read_json(selected_json_path)
    
    
    for items in selected_json['analyticsInfo']:
        hold        = True
        object_name = items['objectImage']
        series_name = '_'.join(object_name.split('_')[0:2])
        vid_path    = f"{video_root}/{'_'.join(object_name.split('_')[0:2])}.mp4"

        try:
            if items['recommended_ads'] != {}:
                display_adds(items, object_name, series_name, vid_path, display_color=True, display_pattern=True)

            else:
                 
                obj_path    = f'{images_root_path}/{series_name}/multi_color/{object_name}'
                if os.path.isfile(obj_path):
                    display_adds(items, object_name, series_name, vid_path, display_color=True, display_pattern=False)

                else:
                    display_adds(items, object_name, series_name, vid_path, display_color=False, display_pattern=False)
        except Exception as e:
            print(f"{e}")
            continue
            
    done = widgets.Button(description = 'Done')  
    done.on_click(functools.partial(Done, selected_json_path))
    output = widgets.Output()
    display(widgets.VBox([done]), output)

In [3]:
version          = 'sharechat_1.2.1'
client           = 'sharechat'

video_root       = f'/notebooks/common-dataset/common/{client}/video_segmented'
images_root_path = f'/notebooks/common-dataset/common/{client}/{version}/Suggested_ads'
result_path      = f'/notebooks/common-dataset/common/{client}/{version}/result.txt'
json_file_root   = f'/notebooks/common-dataset/common/{client}/{version}/json'
detected_path    = f'/notebooks/common-dataset/common/{client}/{version}/temp-media-folder/analytics'
QA_complted_list = f'/notebooks/common-dataset/common/{client}/{version}/qa_complted.txt'

json_file_paths  = glob(f'{json_file_root}/*.json')



In [4]:
len(glob(json_file_root+'/*'))

492

In [5]:
with open(QA_complted_list, 'a') as f:
    pass
username = input("Username:")
analysis(json_file_paths, result_path, QA_complted_list, username=username)

Username:sangam
492
/notebooks/common-dataset/common/sharechat/video_segmented/39b1e51_1633099897846.mp4
--> 39b1e51_1633099897846_0_F_H11S4V8_shoes.jpg
--> analytics


/notebooks/common-dataset/common/sharechat/video_segmented/39b1e51_1633099897846.mp4
--> 39b1e51_1633099897846_2_M_H11S8V8_jeans_pant.jpg
--> analytics


/notebooks/common-dataset/common/sharechat/video_segmented/39b1e51_1633099897846.mp4
--> 39b1e51_1633099897846_0_F_H12S6V5_shoes.jpg
--> analytics


/notebooks/common-dataset/common/sharechat/video_segmented/39b1e51_1633099897846.mp4
--> 39b1e51_1633099897846_1_F_H14S10V2_full_cami_tops.jpg
--> analytics


/notebooks/common-dataset/common/sharechat/video_segmented/39b1e51_1633099897846.mp4
--> 39b1e51_1633099897846_2_F_H11S11V8_jeans_pant.jpg
--> analytics


/notebooks/common-dataset/common/sharechat/sharechat_1.2.1/Suggested_ads/39b1e51_1633099897846/multi_color_image_matching/39b1e51_1633099897846_1_M_H12S3V8_half_tshirt.jpg/*
/notebooks/common-dataset/common/sharechat/sharechat_1.2.1/Suggested_ads/39b1e51_1633099897846/multi_color_image_matching/39b1e51_1633099897846_1_M_H12S3V8_half_tshirt.jpg/4768b9a4-9e7b-4d43-8e9c-efbfaab9a64d.jpg
/notebooks/common-dataset/common/sharechat/video_segmented/39b1e51_1633099897846.mp4
--> 4768b9a4-9e7b-4d43-8e9c-efbfaab9a64d.jpg
--> 39b1e51_1633099897846_1_M_H12S3V8_half_tshirt.jpg


Output()

In [ ]:
import matplotlib.pyplot as plt
img = plt.imread('/notebooks/common-dataset/common/sharechat/sharechat_1.2.1/Suggested_ads/39a3577_1631803594210/multi_color_image_matching/39a3577_1631803594210_1_M_H16S12V1_half_tshirt.jpg')
plt.imshow()

In [ ]:
glob('/notebooks/common-dataset/common/sharechat/sharechat_1.2.1/Suggested_ads/*/multi_color_image_matching/*')